In [1]:
#James Schallert
#Data Mining
#Homework 5

In [2]:
import numpy as np

In [3]:
def  mybLDA_train(xN,xP):

    print("X Positive Matrix")
    print(np.matrix(xP))
    print('\n')

    print("X Negative Matrix")
    print(np.matrix(xN))
    print('\n')
    
    ##Calculate means.
    xPMean = np.mean(xP,axis=1)
    xNMean = np.mean(xN,axis=1)
    print("\n X Positive Mean:")
    print(xPMean)
    print("X Negative Mean:")
    print(xNMean)
    ##Make means into a column vector.
    xPMean = np.reshape(xPMean,(3,1))
    xNMean = np.reshape(xNMean,(3,1))
    
    ##Center the Matrices by subtracting their means.
    xPCentered = np.subtract(xP,xPMean) 
    xNCentered = np.subtract(xN,xNMean)
    
    ## Calculate covariance matrices (X^t)*X
    xPCov = np.matmul(xP,np.transpose(xPCentered ))/len(xPCentered [0])
    xNCov = np.matmul(xN,np.transpose(xNCentered))/len(xNCentered[0])
    print("\nX Positive covariance matrix: ")
    print(xPCov)
    print("X Negative covariance matrix: ")
    print(xNCov)
    
    ##Calculate sB
    meanDifference = (xPMean-xNMean)
    sBetween = np.matmul(meanDifference,np.transpose(meanDifference))
    
    print("\nSB: ")
    print(sBetween)
    
    ##Calculate sW
    n = len(xP[0])+len(xN[0])
    sWithin = len(xP[0])*xPCov/n + len(xN[0])*xNCov/n
    
    print("\nSW: ")
    print(sWithin)
    
    ## Solve for V = (sW^-1 * sB), then calculate its eiganvalues
    sWInv = np.linalg.inv(sWithin)
    eig = np.linalg.eig(np.matmul(sWInv,sBetween))
    print("\nEigan Values and Vectors:")
    print(eig)
    ##Find the largest eiganvalue and its corresponding eiganvector
    biggestEiganValue = np.max(eig[0])
    biggestEiganIndex = np.where(eig[0]==biggestEiganValue)
    modelVector = eig[1][:,biggestEiganIndex]
    
    modelVector = np.reshape(modelVector,(3,1))

    ##Calculate a midpoint for the threshold of the classifier by finding the max projection of xN and min of xP then
    ##finding the mean.
    if np.mean(np.matmul(np.transpose(xN),modelVector)) < np.mean(np.matmul(np.transpose(xP),modelVector)):
        xNMax = max(np.matmul(np.transpose(xN),modelVector))
        xPMin = min(np.matmul(np.transpose(xP),modelVector))
        thresholdValue = (xNMax+xPMin)/2
    else:
        xNMin = min(np.matmul(np.transpose(xN),modelVector))
        xPMax = max(np.matmul(np.transpose(xP),modelVector))
        thresholdValue = (xPMax+xNMin)/2
        
    ##I'm not sure how to find a threshold value without the model, so I'm just appending it and passing it as well.
    
    modelVectorAndThreshold = np.append(modelVector,thresholdValue)
    print("\n Model Vector:")
    print(modelVectorAndThreshold[0:3])
    print("\n Model Threshold:")
    print(modelVectorAndThreshold[3])
    
    return modelVectorAndThreshold


In [4]:
def mybLDA_classify(matX, modelVectorAndThreshold):

    ##Multipy by the modelVector
    modeledX = np.matmul(np.transpose(matX),modelVectorAndThreshold[0:3])
    
    ##Subtract the Threshold
    classifiedX = modeledX-modelVectorAndThreshold[3]
    
    ##Negative values unintuitively  are "positive" 1, positive values are "negative" 1.
    for i in range(0,len(classifiedX)):
        if classifiedX[i] < 0:
            classifiedX[i] = 1
        else:
            classifiedX[i] = -1
            
    
    print("\n Test Data")
    print(np.matrix(matX))
    print("\n Modeled Data: ")
    print(modeledX)
    print("\n Classification:")
    print(classifiedX)
        
    return classifiedX

    
    


In [5]:
##Solving for the classifier and threshold, then checking it on the original data.

xP =[[4,2,2,3,4,6,3,8],
    [1,4,3,6,4,2,2,3],
    [0,1,1,0,-1,0,1,0]]

xN =[[9,6,9, 8, 10],
    [10, 8, 5, 7, 8],
    [1, 0, 0, 1, -1]]

modelVectorAndThreshold = mybLDA_train(xN,xP)

##Concatenating xP and xN to test the model vector and threshold
rows = len(xP)
xAll = xP
for i in range(0,rows):
     xAll[i] = np.concatenate((xAll[i],xN[i]))

classifiedArray = mybLDA_classify(xAll,modelVectorAndThreshold)

X Positive Matrix
[[ 4  2  2  3  4  6  3  8]
 [ 1  4  3  6  4  2  2  3]
 [ 0  1  1  0 -1  0  1  0]]


X Negative Matrix
[[ 9  6  9  8 10]
 [10  8  5  7  8]
 [ 1  0  0  1 -1]]



 X Positive Mean:
[4.    3.125 0.25 ]
X Negative Mean:
[8.4 7.6 0.2]

X Positive covariance matrix: 
[[ 3.75     -0.75     -0.625   ]
 [-0.75      2.109375 -0.15625 ]
 [-0.625    -0.15625   0.4375  ]]
X Negative covariance matrix: 
[[ 1.84 -0.04 -0.28]
 [-0.04  2.64  0.28]
 [-0.28  0.28  0.56]]

SB: 
[[ 1.9360000e+01  1.9690000e+01 -2.2000000e-01]
 [ 1.9690000e+01  2.0025625e+01 -2.2375000e-01]
 [-2.2000000e-01 -2.2375000e-01  2.5000000e-03]]

SW: 
[[ 3.01538462 -0.47692308 -0.49230769]
 [-0.47692308  2.31346154  0.01153846]
 [-0.49230769  0.01153846  0.48461538]]

Eigan Values and Vectors:
(array([2.00457862e+01, 1.26475434e-15, 2.06366329e-18]), array([[ 0.56941154,  0.70555102,  0.00629556],
       [ 0.62283144, -0.69525867,  0.00498278],
       [ 0.53651793, -0.13716099,  0.99996777]]))

 Model Vector:
[0.5

In [6]:
##Solving for the classifier and threshold, then using it to classify the test data.

setX = [[1.3,2.4,6.7,2.2,3.4,3.2],
       [8.1,7.6,2.1,1.1,0.5,7.4],
        [-1,2,3,-2,0,2]]

xP =[[4,2,2,3,4,6,3,8],
    [1,4,3,6,4,2,2,3],
    [0,1,1,0,-1,0,1,0]]

xN =[[9,6,9, 8, 10],
    [10, 8, 5, 7, 8],
    [1, 0, 0, 1, -1]]

modelVectorAndThreshold = mybLDA_train(xN,xP)

classifiedArray = mybLDA_classify(setX,modelVectorAndThreshold)

X Positive Matrix
[[ 4  2  2  3  4  6  3  8]
 [ 1  4  3  6  4  2  2  3]
 [ 0  1  1  0 -1  0  1  0]]


X Negative Matrix
[[ 9  6  9  8 10]
 [10  8  5  7  8]
 [ 1  0  0  1 -1]]



 X Positive Mean:
[4.    3.125 0.25 ]
X Negative Mean:
[8.4 7.6 0.2]

X Positive covariance matrix: 
[[ 3.75     -0.75     -0.625   ]
 [-0.75      2.109375 -0.15625 ]
 [-0.625    -0.15625   0.4375  ]]
X Negative covariance matrix: 
[[ 1.84 -0.04 -0.28]
 [-0.04  2.64  0.28]
 [-0.28  0.28  0.56]]

SB: 
[[ 1.9360000e+01  1.9690000e+01 -2.2000000e-01]
 [ 1.9690000e+01  2.0025625e+01 -2.2375000e-01]
 [-2.2000000e-01 -2.2375000e-01  2.5000000e-03]]

SW: 
[[ 3.01538462 -0.47692308 -0.49230769]
 [-0.47692308  2.31346154  0.01153846]
 [-0.49230769  0.01153846  0.48461538]]

Eigan Values and Vectors:
(array([2.00457862e+01, 1.26475434e-15, 2.06366329e-18]), array([[ 0.56941154,  0.70555102,  0.00629556],
       [ 0.62283144, -0.69525867,  0.00498278],
       [ 0.53651793, -0.13716099,  0.99996777]]))

 Model Vector:
[0.5